# Projeto Módulo I - "3MA Encrypter"

- Criptografia de mensagens com chave única

### Grupo:
- Ana Carolina
- Maria Luísa
- Marília
- Mayara

#### Sobre o projeto:

Inicialmente pensamos em desenvolver um código com chave simples. Pensamos em transformar uma chave string em números decimais de referência da tabela ASCII. No caso, dos caracteres válidos (alfanuméricos e caracteres especiais, 32 a 126), tomando o cuidado de manter o número sempre dentro deste intervalo.

Desenvolvemos o primeiro bloco de código que se tornou o Menu do programa, com um loop e opções para codificação, decodificação e para encerrar o programa.

A chave gerava um número que era somado a cada caractere da mensagem, gerando um novo caractere. Porém, lendo a respeito, percebemos que é uma chave facilmente quebrável. Para isso, resolvemos tornar o processo de codificação um pouco mais complexo:

- incrementamos a chave para corresponder ao tamanho da frase/mensagem
- somamos o número ASCII da frase e chave de acordo com o índice correspondente de cada (frase[i] + chave[i + 1], frase[i+1] + chave[i+2],...)
- inserimos caracteres aleatórios a cada 3 elementos da frase.

Para finalizar, criamos funções que englobavam blocos de códigos específicos, simplificando a leitura final do código, e documentando tais funções.

### Instalar e importar bibliotecas necessárias:

In [ ]:
# Import bibliotecas necessárias

import random
import unicodedata

### Função de tratamento de acentuação nas strings de entrada

### Função de loop de intervalo ASCII (32 a 126)

In [ ]:
# Tratamento de acentuação nas frases e chave. Usado nas funções validar_chave() codificar()
def remover_acentos(frase:str) -> str:
    '''
    Substitui caracteres com acentos pelo seu equivalente sem acentos. Caso a substitução não seja possível e o caractere
    não esteja na tabela ASCII, é removido da string.
    '''
    frase_sem_acentos = unicodedata.normalize('NFKD', frase).encode('ASCII', 'ignore').decode('ASCII')
    return frase_sem_acentos


# Mantem os caracteres dentro dos valores de referência 32 a 126 da tabela ascii, com caracteres alfanuméricos e símbolos especiais
def adaptar_valor_ascii(valor:int) -> int:
    '''
    Recebe um valor numérico inteiro e retorna sua posição equivalente nos caracteres válidos da tabela ASCII.
    Usa como referência de caracteres válidos os valores de 32 a 126. Caso seja maior que 126, retorna para o 32.
    '''
    valor_ascii_inicial = 32
    valor_ascii_final = 126
    if valor > valor_ascii_final:
        valor -= 95
        valor = adaptar_valor_ascii(valor)
    elif valor < valor_ascii_inicial:
        valor += 95
        valor = adaptar_valor_ascii(valor)
        
    return valor

### Funções referentes a chave:

In [ ]:
# Funções de chave
# Validar a chave, garante que a chave tenha tamanho entre 5 e 10 caracteres

def validar_chave() -> str:
    '''
    Registra e valida o valor da chave inserida pelo usuário, garantindo que tenha entre 5 e 10 caracteres válidos.
    Remove acentos e caracteres fora da tabela ASCII usando a função remover_acentos()
    '''
    chave = input("Insira a chave secreta: ")
    chave = remover_acentos(chave)
    if len(chave) < 5 or len(chave) > 10:
        print("A chave deve ter entre 5 e 10 caracteres válidos.")
        chave = validar_chave()
    return chave


# Para embaralhar e aumentar a complexidade da codificação, igualamos o tamanho da chave ao da frase a ser codificada/decodificada para
# somar um valor diferente para cada caractere da mensagem.

# Em casos que a chave seja maior que a frase, função para diminuir o número de caracteres da chave, se tornando igual à frase
def diminuir_chave(chave:list, tamanho_frase:int) -> list:
    '''
    Recebe uma lista de valores inteiros referentes ao código ASCII de caracteres e retorna uma lista atualizada do mesmo tamanho da frase.
    Calcula novos valores para todos os caracteres, somando o primeiro com o último, segundo com penúltimo e assim sucessivamente.
    '''
    contador = -1
    nova_chave = []
    for i in range(tamanho_frase):
        novo_valor = chave[i] + chave[contador]
        # Garante que o valor ainda estará entre os caracteres válidos (32 - 126).
        if novo_valor > 126:
            novo_valor = adaptar_valor_ascii(novo_valor)
        nova_chave.append(novo_valor)
        contador -= 1
    return nova_chave

# Função para incrementar chave, funciona de maneira oposta à anterior, repetindo os caracteres da chave, para que tenham
# o mesmo tamanho da frase
def incrementar_chave(chave:list, tamanho_frase:int) -> list:
    '''
        Aumenta a chave repetindo os caracteres até que tenha o mesmo tamanho da frase.
        Ex.: 
        frase = "Frase teste 1" - 13 caracteres
        chave = "abc12" - 5 caracteres
        chave_final = "abc12abc12abc" - 13 caracteres
    '''
    contador = 0
    chave_final = chave
    # Repete a chave adicionando caracteres até que tenha o mesmo tamanho da frase.
    while len(chave_final) < tamanho_frase:
        if contador == len(chave):
            contador = 0
        else:
            chave_final.append(chave[contador])
            contador += 1
    return chave_final

# Função que transforma a string chave em números da tabela ASCII
def calcular_chave(chave:str, frase:str) -> list:
    '''
        Transforma a chave em uma lista de números baseados na tabela ASCII.
        Soma o valor do caractere com seu índice + 1, para que a chave seja única. Ex.: 21 != 12
        Caso a chave seja menor que a frase, chama a função incrementar_chave().
        Caso a chave seja maior que a frase, chama a função diminuir_chave().
    '''
     
    lista_chave = []
    # Soma o código de cada caractere com seu índice para que o valor seja mais único
    for i in range(len(chave)):
        lista_chave.append(ord(chave[i]) + (i + 1))

    # Ajustar chave para o tamanho da frase.
    if len(chave) < len(frase):
        chave = incrementar_chave(lista_chave, len(frase))
    elif len(chave) > len(frase):
        lista_chave = diminuir_chave(lista_chave, len(frase))

    return lista_chave

### Funções de inserção e remoção de caracteres extras para aumentar a complexidade da codificação

In [ ]:
# Inserção (codificação) e retirada (decodificação) de caracteres à frase, aumentando a complexidade da codificação

def inserir_caracteres(frase:list) -> list:
    '''
    Recebe uma lista de caracteres, insere um caracter inicial e adiciona caracteres aleatórios nos valores pares.
    '''
    nova_frase = frase[::]       
    for i in range(0, (len(frase) + (len(frase) // 2) + 1), 3):
        nova_frase.insert(i, chr(random.randint(33, 126)))            
    return nova_frase

def remover_caracteres(frase:list) -> list:
    '''
    Recebe uma lista de caracteres e remove os caracteres de índices múltiplos de 3.
    '''
    nova_frase = []
    for i in range(len(frase)):
        if i % 3 != 0:
            nova_frase.append(frase[i])
    return nova_frase

### Funções principais do código:
- Codificação
- Decodificação

In [ ]:
def codificar(frase:str, chave:str) -> str:
    '''
        Gera a frase codificada usando a chave fornecida pelo usuário. Soma o valor cada caractere da frase ao valor do caractere 
        de mesmo índice na chave, calculando de acordo com os valores válidos da tabela ASCII.
        Remove os acentos usando a função remover_acentos()
    '''
    frase_final = []
    # Remove acentos e caracteres que não estão na tabela ASCII
    frase = list(remover_acentos(frase))
    # Adapta e transforma a chave
    chave = calcular_chave(chave, frase)
    
    for i in range(len(frase)):
        novo_char = ord(frase[i]) + chave[i]  
        if novo_char > 126:
            # Calcula o novo valor ASCII do caractere e garante que o valor máximo válido seja o 126, que é o '~'.
            novo_char = adaptar_valor_ascii(novo_char)
        frase_final.append(chr(novo_char))
    
    #Insere caracteres aleatórios
    frase_final = inserir_caracteres(frase_final)
    frase_final = "".join(frase_final)
    return frase_final

def decodificar(frase:str, chave:str) -> str:
    '''
        Gera a frase decodificada usando a chave fornecida pelo usuário. Diminui do valor de cada caractere da frase 
        o valor do caractere de mesmo índice da chave, calculando de acordo com os valores da tabela ASCII.
    '''
    frase_final = []
    frase = list(frase)
    # Remove caracteres extras
    frase = remover_caracteres(frase)
    chave = calcular_chave(chave, frase)

    for i in range(len(frase)):
        novo_char = ord(frase[i]) - chave[i]           
        if novo_char < 32:
            # Calcula o novo valor ASCII do caractere e garante que o valor mínimo seja o 32, que é o ' '.
            # Caso seja menor que 32, retorna para o último válido, 126, o caractere '~'.                
            novo_char = adaptar_valor_ascii(novo_char)
        frase_final.append(chr(novo_char))
    
    frase_final = "".join(frase_final)
    return frase_final

### Função extra no menu para gerar chaves aleatórias

In [ ]:
def gerar_chave(tamanho_chave:int) -> str:
    '''
    Gera uma chave aleatória com tamanho especificado pelo usuário, entre 5 e 10 caracteres.
    '''
    chave = []
    for i in range(tamanho_chave):
        chave.append(chr(random.randint(33, 126)))
    chave_final = ''.join(chave)
    return chave_final

### Bloco de código principal do programa

In [ ]:
bright_cyan = "\033[0;96m"
white = "\033[0;37m"
titulo = '''
         `7MMM.     ,MMF'      db          `7MM"""YMM                                                     mm                   
           MMMb    dPMM       ;MM:           MM    `7                                                     MM                   
 pd""b.    M YM   ,M MM      ,V^MM.          MM   d    `7MMpMMMb.  ,p6"bo `7Mb,od8 `7M'   `MF'`7MMpdMAo.mmMMmm .gP"Ya `7Mb,od8 
(O)  `8b   M  Mb  M' MM     ,M  `MM          MMmmMM      MM    MM 6M'  OO   MM' "'   VA   ,V    MM   `Wb  MM  ,M'   Yb  MM' "' 
     ,89   M  YM.P'  MM     AbmmmqMA         MM   Y  ,   MM    MM 8M        MM        VA ,V     MM    M8  MM  8M""""""  MM     
   ""Yb.   M  `YM'   MM    A'     VML        MM     ,M   MM    MM YM.    ,  MM         VVV      MM   ,AP  MM  YM.    ,  MM     
      88 .JML. `'  .JMML..AMA.   .AMMA.    .JMMmmmmMMM .JMML  JMML.YMbmd' .JMML.       ,V       MMbmmd'   `Mbmo`Mbmmd'.JMML.   
(O)  .M'                                                                              ,V        MM                             
 bmmmd'                                                                            OOb"       .JMML.                             
 '''
print("-------------------------------------------------------------------------------------------------------------------------------")
print(bright_cyan + titulo)
print(white + "-------------------------------------------------------------------------------------------------------------------------------\n")
print("Bem vindo! Selecione a opção que você deseja realizar: ")

while True:
    opcao = input("1. Codificar \n2. Decodificar \n3. Gerar nova chave aleatória \n4. Sair \nDigite o número da opção desejada: ").capitalize()
    if opcao == "1" or opcao == "Codificar":
        frase = input("Informe a mensagem a ser codificada: ")
        chave = validar_chave()  
        frase_codificada = codificar(frase, chave)
        print(f"Frase codificada: {frase_codificada}")
        
    elif opcao == "2" or opcao == "Decodificar":
        frase = input("Informe a mensagem a ser decodificada: ")
        chave = validar_chave()
        frase_decodificada = decodificar(frase, chave)
        print(f"Frase decodificada: {frase_decodificada}")
        
    elif opcao == '3' or opcao == 'Gerar chave':
        tamanho_chave = (int(input("Informe o número de caracteres da chave: ")))
        # Garante que o tamanho da chave gerada será maior que 5 e menor que 10.
        while tamanho_chave < 5 or tamanho_chave > 10:
            print("O tamanho da chave deve ser entre 5 e 10.")
            tamanho_chave = (int(input("Informe o número de caracteres da chave: ")))
        nova_chave = gerar_chave(tamanho_chave)
        print(f"Chave: {nova_chave}")
        
    elif opcao == "4" or opcao == "Sair":
        print("Encerrando.")
        break
    
    else:
        print("Opção inválida, tente novamente.")

### Todo o código do projeto concentrado numa única célula

In [ ]:
# Import bibliotecas necessárias

import random
import unicodedata


# Tratamento de acentuação nas frases e chave. Usado nas funções validar_chave() codificar()

def remover_acentos(frase:str) -> str:
    '''
    Substitui caracteres com acentos pelo seu equivalente sem acentos. Caso a substitução não seja possível e o caractere
    não esteja na tabela ASCII, é removido da string.
    '''
    frase_sem_acentos = unicodedata.normalize('NFKD', frase).encode('ASCII', 'ignore').decode('ASCII')
    return frase_sem_acentos


# Mantem os caracteres dentro dos valores de referência 32 a 126 da tabela ascii, com caracteres alfanuméricos e símbolos especiais

def adaptar_valor_ascii(valor:int) -> int:
    '''
    Recebe um valor numérico inteiro e retorna sua posição equivalente nos caracteres válidos da tabela ASCII.
    Usa como referência de caracteres válidos os valores de 32 a 126. Caso seja maior que 126, retorna para o 32.
    '''
    valor_ascii_inicial = 32
    valor_ascii_final = 126
    if valor > valor_ascii_final:
        valor -= 95
        valor = adaptar_valor_ascii(valor)
    elif valor < valor_ascii_inicial:
        valor += 95
        valor = adaptar_valor_ascii(valor)
        
    return valor


# Funções de chave
# Validar a chave, garante que a chave tenha tamanho entre 5 e 10 caracteres

def validar_chave() -> str:
    '''
    Registra e valida o valor da chave inserida pelo usuário, garantindo que tenha entre 5 e 10 caracteres válidos.
    Remove acentos e caracteres fora da tabela ASCII usando a função remover_acentos()
    '''
    chave = input("Insira a chave secreta: ")
    chave = remover_acentos(chave)
    if len(chave) < 5 or len(chave) > 10:
        print("A chave deve ter entre 5 e 10 caracteres válidos.")
        chave = validar_chave()
    return chave


# Para embaralhar e aumentar a complexidade da codificação, igualamos o tamanho da chave ao da frase a ser codificada/decodificada para
# somar um valor diferente para cada caractere da mensagem.

# Em casos que a chave seja maior que a frase, função para diminuir o número de caracteres da chave, se tornando igual à frase
def diminuir_chave(chave:list, tamanho_frase:int) -> list:
    '''
    Recebe uma lista de valores inteiros referentes ao código ASCII de caracteres e retorna uma lista atualizada do mesmo tamanho da frase.
    Calcula novos valores para todos os caracteres, somando o primeiro com o último, segundo com penúltimo e assim sucessivamente.
    '''
    contador = -1
    nova_chave = []
    for i in range(tamanho_frase):
        novo_valor = chave[i] + chave[contador]
        # Garante que o valor ainda estará entre os caracteres válidos (32 - 126).
        if novo_valor > 126:
            novo_valor = adaptar_valor_ascii(novo_valor)
        nova_chave.append(novo_valor)
        contador -= 1
    return nova_chave

# Função para incrementar chave, funciona de maneira oposta à anterior, repetindo os caracteres da chave, para que tenham
# o mesmo tamanho da frase
def incrementar_chave(chave:list, tamanho_frase:int) -> list:
    '''
        Aumenta a chave repetindo os caracteres até que tenha o mesmo tamanho da frase.
        Ex.: 
        frase = "Frase teste 1" - 13 caracteres
        chave = "abc12" - 5 caracteres
        chave_final = "abc12abc12abc" - 13 caracteres
    '''
    contador = 0
    chave_final = chave
    # Repete a chave adicionando caracteres até que tenha o mesmo tamanho da frase.
    while len(chave_final) < tamanho_frase:
        if contador == len(chave):
            contador = 0
        else:
            chave_final.append(chave[contador])
            contador += 1
    return chave_final

def calcular_chave(chave:str, frase:str) -> list:
    '''
        Transforma a chave em uma lista de números baseados na tabela ASCII.
        Soma o valor do caractere com seu índice + 1, para que a chave seja única. Ex.: 21 != 12
        Caso a chave seja menor que a frase, chama a função incrementar_chave().
        Caso a chave seja maior que a frase, chama a função diminuir_chave().
    '''
    
    lista_chave = []
    # Soma o código de cada caracter com seu índice para que o valor seja mais único
    for i in range(len(chave)):
        lista_chave.append(ord(chave[i]) + (i + 1))
    
    # Ajustar chave para o tamanho da frase.
    if len(chave) < len(frase):
        chave = incrementar_chave(lista_chave, len(frase))
    elif len(chave) > len(frase):
        lista_chave = diminuir_chave(lista_chave, len(frase))

    return lista_chave


# Inserção (codificação) e retirada (decodificação) de caracteres à frase, aumentando a complexidade da codificação

def inserir_caracteres(frase:list) -> list:
    '''
    Recebe uma lista de caracteres, insere um caracter inicial e adiciona caracteres aleatórios nos valores pares.
    '''
    nova_frase = frase[::]       
    for i in range(0, (len(frase) + (len(frase) // 2) + 1), 3):
        nova_frase.insert(i, chr(random.randint(33, 126)))            
    return nova_frase

def remover_caracteres(frase:list) -> list:
    '''
    Recebe uma lista de caracteres e remove os caracteres de índices múltiplos de 3.
    '''
    nova_frase = []
    for i in range(len(frase)):
        if i % 3 != 0:
            nova_frase.append(frase[i])
    return nova_frase


# Funções principais para codificação e decodificação da mensagem

def codificar(frase:str, chave:str) -> str:
    '''
        Gera a frase codificada usando a chave fornecida pelo usuário. Soma o valor cada caractere da frase ao valor do caractere 
        de mesmo índice na chave, calculando de acordo com os valores válidos da tabela ASCII.
        Remove os acentos usando a função remover_acentos()
    '''
    frase_final = []
    # Remove acentos e caracteres que não estão na tabela ASCII
    frase = list(remover_acentos(frase))
    # Adapta e transforma a chave
    chave = calcular_chave(chave, frase)
    
    for i in range(len(frase)):
        novo_char = ord(frase[i]) + chave[i]  
        if novo_char > 126:
            # Calcula o novo valor ASCII do caractere e garante que o valor máximo válido seja o 126, que é o '~'.
            novo_char = adaptar_valor_ascii(novo_char)
        frase_final.append(chr(novo_char))
    #Insere caracteres aleatórios
    frase_final = inserir_caracteres(frase_final) 
    frase_final = "".join(frase_final)
    return frase_final

def decodificar(frase:str, chave:str) -> str:
    '''
        Gera a frase decodificada usando a chave fornecida pelo usuário. Diminui do valor de cada caractere da frase 
        o valor do caractere de mesmo índice da chave, calculando de acordo com os valores da tabela ASCII.
    '''
    frase_final = []
    frase = list(frase)
    # Remove os caracteres adicionados na codificação
    frase = remover_caracteres(frase) 
    chave = calcular_chave(chave, frase)

    for i in range(len(frase)):
        novo_char = ord(frase[i]) - chave[i]           
        if novo_char < 32:
            # Calcula o novo valor ASCII do caractere e garante que o valor mínimo seja o 32, que é o ' '.
            # Caso seja menor que 32, retorna para o último válido, 126, o caractere '~'.                
            novo_char = adaptar_valor_ascii(novo_char)
        frase_final.append(chr(novo_char))
    frase_final = "".join(frase_final)
    return frase_final       

def gerar_chave(tamanho_chave:int) -> str:
    '''
    Gera uma chave aleatória com tamanho especificado pelo usuário, entre 5 e 10 caracteres.
    '''
    chave = []
    for i in range(tamanho_chave):
        chave.append(chr(random.randint(33, 126)))
    chave_final = ''.join(chave)
    return chave_final

    
bright_cyan = "\033[0;96m"
white = "\033[0;37m"
titulo = '''
         `7MMM.     ,MMF'      db          `7MM"""YMM                                                     mm                   
           MMMb    dPMM       ;MM:           MM    `7                                                     MM                   
 pd""b.    M YM   ,M MM      ,V^MM.          MM   d    `7MMpMMMb.  ,p6"bo `7Mb,od8 `7M'   `MF'`7MMpdMAo.mmMMmm .gP"Ya `7Mb,od8 
(O)  `8b   M  Mb  M' MM     ,M  `MM          MMmmMM      MM    MM 6M'  OO   MM' "'   VA   ,V    MM   `Wb  MM  ,M'   Yb  MM' "' 
     ,89   M  YM.P'  MM     AbmmmqMA         MM   Y  ,   MM    MM 8M        MM        VA ,V     MM    M8  MM  8M""""""  MM     
   ""Yb.   M  `YM'   MM    A'     VML        MM     ,M   MM    MM YM.    ,  MM         VVV      MM   ,AP  MM  YM.    ,  MM     
      88 .JML. `'  .JMML..AMA.   .AMMA.    .JMMmmmmMMM .JMML  JMML.YMbmd' .JMML.       ,V       MMbmmd'   `Mbmo`Mbmmd'.JMML.   
(O)  .M'                                                                              ,V        MM                             
 bmmmd'                                                                            OOb"       .JMML.                             
 '''
print("-------------------------------------------------------------------------------------------------------------------------------")
print(bright_cyan + titulo)
print(white + "-------------------------------------------------------------------------------------------------------------------------------\n")
print("Bem vindo! Selecione a opção que você deseja realizar: ")

while True:
    opcao = input("1. Codificar \n2. Decodificar \n3. Gerar nova chave aleatória \n4. Sair \nDigite o número da opção desejada: ").capitalize()

    if opcao == "1" or opcao == "Codificar":
        frase = input("Informe a mensagem a ser codificada: ")
        chave = validar_chave()  
        frase_codificada = codificar(frase, chave)
        print(f"Frase codificada: {frase_codificada}")
        
    elif opcao == "2" or opcao == "Decodificar":
        frase = input("Informe a mensagem a ser decodificada: ")
        chave = validar_chave()
        frase_decodificada = decodificar(frase, chave)
        print(f"Frase decodificada: {frase_decodificada}")
        
    elif opcao == '3' or opcao == 'Gerar chave':
        tamanho_chave = (int(input("Informe o número de caracteres da chave: ")))
        # Garante que o tamanho da chave gerada será maior que 5 e menor que 10.
        while tamanho_chave < 5 or tamanho_chave > 10:
            print("O tamanho da chave deve ser entre 5 e 10.")
            tamanho_chave = (int(input("Informe o número de caracteres da chave: ")))
        nova_chave = gerar_chave(tamanho_chave)
        print(f"Chave: {nova_chave}")

    elif opcao == "4" or opcao == "Sair":
        print("Encerrando.")
        break
    
    else:
        print("Opção inválida, tente novamente.")

-------------------------------------------------------------------------------------------------------------------------------

         `7MMM.     ,MMF'      db          `7MM"""YMM                                                     mm                   
           MMMb    dPMM       ;MM:           MM    `7                                                     MM                   
 pd""b.    M YM   ,M MM      ,V^MM.          MM   d    `7MMpMMMb.  ,p6"bo `7Mb,od8 `7M'   `MF'`7MMpdMAo.mmMMmm .gP"Ya `7Mb,od8 
(O)  `8b   M  Mb  M' MM     ,M  `MM          MMmmMM      MM    MM 6M'  OO   MM' "'   VA   ,V    MM   `Wb  MM  ,M'   Yb  MM' "' 
     ,89   M  YM.P'  MM     AbmmmqMA         MM   Y  ,   MM    MM 8M        MM        VA ,V     MM    M8  MM  8M""""""  MM     
   ""Yb.   M  `YM'   MM    A'     VML        MM     ,M   MM    MM YM.    ,  MM         VVV      MM   ,AP  MM  YM.    ,  MM     
      88 .JML. `'  .JMML..AMA.   .AMMA.    .JMMmmmmMMM .JMML  JMML.YMbmd' .JMML.       ,V       MMbmmd'